<a href="https://colab.research.google.com/github/jeremiahoclark/python-coding-patterns/blob/main/06_testing_patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing Patterns in Python

This notebook covers essential testing patterns in Python, including test structure, fixtures, and mocking techniques for writing maintainable and reliable tests.

## 1. Arrange-Act-Assert (AAA) Pattern

The AAA pattern provides a clear structure for tests: **Arrange** (set up), **Act** (execute), **Assert** (verify).

In [ ]:
# First, let's define some simple functions to test
def multiply(a, b):
    """Simple multiplication function"""
    return a * b

def factorial(n):
    """Calculate factorial of n"""
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    if n == 0 or n == 1:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

class Stack:
    """Simple stack implementation"""
    def __init__(self):
        self._items = []

    def push(self, item):
        self._items.append(item)

    def pop(self):
        if not self._items:
            raise IndexError("pop from empty stack")
        return self._items.pop()

    def peek(self):
        if not self._items:
            raise IndexError("peek from empty stack")
        return self._items[-1]

    def size(self):
        return len(self._items)

    def is_empty(self):
        return len(self._items) == 0

In [ ]:
# AAA Pattern Examples using pytest-style asserts
import pytest

def test_multiply_with_positive_numbers():
    """Test multiplication with positive numbers following AAA pattern"""
    # Arrange
    x = 4
    y = 5
    expected = 20

    # Act
    result = multiply(x, y)

    # Assert
    assert result == expected

def test_multiply_with_zero():
    """Test multiplication with zero"""
    # Arrange
    x = 0
    y = 123
    expected = 0

    # Act
    result = multiply(x, y)

    # Assert
    assert result == expected

def test_factorial_base_cases():
    """Test factorial base cases"""
    # Arrange
    test_cases = [(0, 1), (1, 1)]

    for n, expected in test_cases:
        # Act
        result = factorial(n)

        # Assert
        assert result == expected, f"factorial({n}) should be {expected}, got {result}"

def test_factorial_normal_case():
    """Test factorial with normal positive number"""
    # Arrange
    n = 5
    expected = 120  # 5! = 5 * 4 * 3 * 2 * 1

    # Act
    result = factorial(n)

    # Assert
    assert result == expected

def test_factorial_negative_raises_error():
    """Test that factorial raises error for negative numbers"""
    # Arrange
    n = -1

    # Act & Assert (combined for exception testing)
    with pytest.raises(ValueError, match="Factorial is not defined for negative numbers"):
        factorial(n)

# Run the tests
print("Running AAA pattern tests...")
test_multiply_with_positive_numbers()
test_multiply_with_zero()
test_factorial_base_cases()
test_factorial_normal_case()
test_factorial_negative_raises_error()
print("All AAA pattern tests passed!")

In [ ]:
# Stack class testing with AAA pattern
def test_stack_push_and_peek():
    """Test pushing item to stack and peeking"""
    # Arrange
    stack = Stack()
    item = "test_item"

    # Act
    stack.push(item)
    top_item = stack.peek()

    # Assert
    assert top_item == item
    assert stack.size() == 1
    assert not stack.is_empty()

def test_stack_push_and_pop():
    """Test pushing and popping from stack"""
    # Arrange
    stack = Stack()
    item = "test_item"

    # Act
    stack.push(item)
    popped_item = stack.pop()

    # Assert
    assert popped_item == item
    assert stack.size() == 0
    assert stack.is_empty()

def test_stack_empty_operations_raise_errors():
    """Test that operations on empty stack raise appropriate errors"""
    # Arrange
    stack = Stack()

    # Act & Assert
    with pytest.raises(IndexError, match="pop from empty stack"):
        stack.pop()

    with pytest.raises(IndexError, match="peek from empty stack"):
        stack.peek()

def test_stack_multiple_items():
    """Test stack with multiple items (LIFO behavior)"""
    # Arrange
    stack = Stack()
    items = ["first", "second", "third"]

    # Act
    for item in items:
        stack.push(item)

    # Assert
    assert stack.size() == 3
    assert stack.peek() == "third"  # Last in, first out

    # Test popping order
    assert stack.pop() == "third"
    assert stack.pop() == "second"
    assert stack.pop() == "first"
    assert stack.is_empty()

# Run stack tests
print("Running Stack AAA tests...")
test_stack_push_and_peek()
test_stack_push_and_pop()
test_stack_empty_operations_raise_errors()
test_stack_multiple_items()
print("All Stack AAA tests passed!")

## 2. Fixtures and Setup Patterns

Fixtures provide reusable setup and teardown for tests, reducing code duplication and ensuring consistent test environments.

In [ ]:
# First, let's create a more complex class that needs setup
import tempfile
import os
import json
from typing import Dict, Any, List

class Database:
    """Simple file-based database for testing"""
    def __init__(self, file_path: str):
        self.file_path = file_path
        self.data = {}

    def connect(self):
        """Initialize database file"""
        if os.path.exists(self.file_path):
            with open(self.file_path, 'r') as f:
                self.data = json.load(f)
        else:
            self.data = {}
            self.save()

    def disconnect(self):
        """Save and close database"""
        self.save()
        self.data = {}

    def save(self):
        """Persist data to file"""
        with open(self.file_path, 'w') as f:
            json.dump(self.data, f)

    def add(self, key: str, value: Any):
        """Add data to database"""
        self.data[key] = value
        self.save()

    def get(self, key: str, default=None):
        """Get data from database"""
        return self.data.get(key, default)

    def delete(self, key: str):
        """Delete key from database"""
        if key in self.data:
            del self.data[key]
            self.save()

    def count(self):
        """Count number of items"""
        return len(self.data)

    def clear(self):
        """Clear all data"""
        self.data = {}
        self.save()

# Pytest-style fixtures
import pytest

@pytest.fixture
def temp_db_file():
    """Fixture that provides a temporary file path for database"""
    with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
        temp_path = f.name

    yield temp_path

    # Cleanup
    if os.path.exists(temp_path):
        os.unlink(temp_path)

@pytest.fixture
def db(temp_db_file):
    """Fixture that provides a connected database instance"""
    database = Database(temp_db_file)
    database.connect()

    yield database

    database.disconnect()

@pytest.fixture
def populated_db(db):
    """Fixture that provides a database with sample data"""
    sample_data = {
        "user1": {"name": "Alice", "age": 30},
        "user2": {"name": "Bob", "age": 25},
        "user3": {"name": "Charlie", "age": 35}
    }

    for key, value in sample_data.items():
        db.add(key, value)

    return db

In [ ]:
# Manual fixture simulation for demonstration
# (In real pytest, these would be automatically injected)

def create_temp_db():
    """Create temporary database for testing"""
    temp_file = tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False)
    temp_path = temp_file.name
    temp_file.close()

    db = Database(temp_path)
    db.connect()
    return db, temp_path

def cleanup_temp_db(db, temp_path):
    """Clean up temporary database"""
    db.disconnect()
    if os.path.exists(temp_path):
        os.unlink(temp_path)

# Tests using fixtures
def test_db_starts_empty():
    """Test that new database starts empty"""
    db, temp_path = create_temp_db()
    try:
        assert db.count() == 0
    finally:
        cleanup_temp_db(db, temp_path)

def test_db_add_and_get():
    """Test adding and retrieving data"""
    db, temp_path = create_temp_db()
    try:
        # Add data
        db.add("test_key", "test_value")

        # Verify
        assert db.get("test_key") == "test_value"
        assert db.count() == 1
    finally:
        cleanup_temp_db(db, temp_path)

def test_db_persistence():
    """Test that data persists after reconnection"""
    db, temp_path = create_temp_db()
    try:
        # Add data and disconnect
        db.add("persistent_key", "persistent_value")
        db.disconnect()

        # Reconnect and verify data persists
        db.connect()
        assert db.get("persistent_key") == "persistent_value"
        assert db.count() == 1
    finally:
        cleanup_temp_db(db, temp_path)

def test_db_delete():
    """Test deleting data"""
    db, temp_path = create_temp_db()
    try:
        # Add and then delete
        db.add("temp_key", "temp_value")
        assert db.count() == 1

        db.delete("temp_key")
        assert db.count() == 0
        assert db.get("temp_key") is None
    finally:
        cleanup_temp_db(db, temp_path)

def test_db_with_sample_data():
    """Test operations on populated database"""
    db, temp_path = create_temp_db()
    try:
        # Populate with sample data (like populated_db fixture)
        sample_data = {
            "user1": {"name": "Alice", "age": 30},
            "user2": {"name": "Bob", "age": 25},
            "user3": {"name": "Charlie", "age": 35}
        }

        for key, value in sample_data.items():
            db.add(key, value)

        # Test operations
        assert db.count() == 3
        assert db.get("user1")["name"] == "Alice"
        assert db.get("user2")["age"] == 25

        # Test deletion
        db.delete("user2")
        assert db.count() == 2
        assert db.get("user2") is None
    finally:
        cleanup_temp_db(db, temp_path)

# Run fixture tests
print("Running fixture pattern tests...")
test_db_starts_empty()
test_db_add_and_get()
test_db_persistence()
test_db_delete()
test_db_with_sample_data()
print("All fixture pattern tests passed!")

In [ ]:
# unittest-style fixtures using setUp and tearDown
import unittest

class TestDatabaseWithSetUp(unittest.TestCase):
    """Example of unittest-style fixtures"""

    def setUp(self):
        """Set up test fixtures before each test method"""
        self.temp_file = tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False)
        self.temp_path = self.temp_file.name
        self.temp_file.close()

        self.db = Database(self.temp_path)
        self.db.connect()

    def tearDown(self):
        """Clean up after each test method"""
        self.db.disconnect()
        if os.path.exists(self.temp_path):
            os.unlink(self.temp_path)

    def test_starts_empty(self):
        """Test database starts empty"""
        self.assertEqual(self.db.count(), 0)

    def test_add_items(self):
        """Test adding items to database"""
        self.db.add("key1", "value1")
        self.db.add("key2", "value2")

        self.assertEqual(self.db.count(), 2)
        self.assertEqual(self.db.get("key1"), "value1")
        self.assertEqual(self.db.get("key2"), "value2")

    def test_clear_database(self):
        """Test clearing database"""
        self.db.add("temp", "data")
        self.assertEqual(self.db.count(), 1)

        self.db.clear()
        self.assertEqual(self.db.count(), 0)

# Run unittest tests manually
def run_unittest_tests():
    """Manually run unittest-style tests"""
    suite = unittest.TestLoader().loadTestsFromTestCase(TestDatabaseWithSetUp)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)
    return result.wasSuccessful()

print("Running unittest-style fixture tests...")
success = run_unittest_tests()
if success:
    print("All unittest fixture tests passed!")
else:
    print("Some unittest fixture tests failed.")

## 3. Mocking and Patching

Mocking allows testing code in isolation by replacing external dependencies with controlled fake objects.

In [ ]:
# Code that interacts with external services (to be mocked)
import smtplib
import requests
import os
from email.mime.text import MIMEText
from typing import Dict, Any

class EmailService:
    """Email service that we'll test using mocks"""

    def __init__(self, smtp_server: str, port: int = 587):
        self.smtp_server = smtp_server
        self.port = port

    def send_email(self, from_addr: str, to_addr: str, subject: str, body: str) -> bool:
        """Send email via SMTP"""
        try:
            msg = MIMEText(body)
            msg['Subject'] = subject
            msg['From'] = from_addr
            msg['To'] = to_addr

            server = smtplib.SMTP(self.smtp_server, self.port)
            server.starttls()
            server.sendmail(from_addr, [to_addr], msg.as_string())
            server.quit()
            return True
        except Exception:
            return False

class APIClient:
    """HTTP API client that we'll test using mocks"""

    def __init__(self, base_url: str):
        self.base_url = base_url

    def get_user(self, user_id: str) -> Dict[str, Any]:
        """Get user data from API"""
        response = requests.get(f"{self.base_url}/users/{user_id}")
        response.raise_for_status()
        return response.json()

    def create_user(self, user_data: Dict[str, Any]) -> str:
        """Create new user via API"""
        response = requests.post(f"{self.base_url}/users", json=user_data)
        response.raise_for_status()
        return response.json().get('id')

class ConfigService:
    """Service that reads configuration from environment"""

    @staticmethod
    def get_database_url() -> str:
        """Get database URL from environment"""
        return os.getenv('DATABASE_URL', 'sqlite:///default.db')

    @staticmethod
    def get_api_key() -> str:
        """Get API key from environment"""
        api_key = os.getenv('API_KEY')
        if not api_key:
            raise ValueError("API_KEY environment variable is required")
        return api_key

In [ ]:
# Mocking with unittest.mock
from unittest.mock import Mock, patch, MagicMock, call
import unittest

def test_email_service_success():
    """Test successful email sending using mocks"""
    with patch('smtplib.SMTP') as mock_smtp_class:
        # Arrange
        mock_server = MagicMock()
        mock_smtp_class.return_value = mock_server

        email_service = EmailService("smtp.example.com")

        # Act
        result = email_service.send_email(
            "sender@example.com",
            "recipient@example.com",
            "Test Subject",
            "Test Body"
        )

        # Assert
        assert result is True
        mock_smtp_class.assert_called_once_with("smtp.example.com", 587)
        mock_server.starttls.assert_called_once()
        mock_server.sendmail.assert_called_once()
        mock_server.quit.assert_called_once()

        # Check sendmail was called with correct arguments
        call_args = mock_server.sendmail.call_args
        assert call_args[0][0] == "sender@example.com"
        assert call_args[0][1] == ["recipient@example.com"]
        assert "Test Subject" in call_args[0][2]
        assert "Test Body" in call_args[0][2]

def test_email_service_failure():
    """Test email sending failure handling"""
    with patch('smtplib.SMTP') as mock_smtp_class:
        # Arrange - make SMTP raise an exception
        mock_smtp_class.side_effect = Exception("SMTP connection failed")

        email_service = EmailService("smtp.example.com")

        # Act
        result = email_service.send_email(
            "sender@example.com",
            "recipient@example.com",
            "Test Subject",
            "Test Body"
        )

        # Assert
        assert result is False
        mock_smtp_class.assert_called_once()

print("Running email service mock tests...")
test_email_service_success()
test_email_service_failure()
print("Email service mock tests passed!")

In [ ]:
# Mocking HTTP requests
def test_api_client_get_user():
    """Test API client get_user method"""
    with patch('requests.get') as mock_get:
        # Arrange
        mock_response = Mock()
        mock_response.json.return_value = {
            'id': '123',
            'name': 'Test User',
            'email': 'test@example.com'
        }
        mock_response.raise_for_status.return_value = None
        mock_get.return_value = mock_response

        client = APIClient("https://api.example.com")

        # Act
        user = client.get_user("123")

        # Assert
        assert user['id'] == '123'
        assert user['name'] == 'Test User'
        mock_get.assert_called_once_with("https://api.example.com/users/123")
        mock_response.raise_for_status.assert_called_once()

def test_api_client_create_user():
    """Test API client create_user method"""
    with patch('requests.post') as mock_post:
        # Arrange
        mock_response = Mock()
        mock_response.json.return_value = {'id': 'new_user_123'}
        mock_response.raise_for_status.return_value = None
        mock_post.return_value = mock_response

        client = APIClient("https://api.example.com")
        user_data = {'name': 'New User', 'email': 'new@example.com'}

        # Act
        user_id = client.create_user(user_data)

        # Assert
        assert user_id == 'new_user_123'
        mock_post.assert_called_once_with(
            "https://api.example.com/users",
            json=user_data
        )

def test_api_client_handles_http_errors():
    """Test API client handles HTTP errors properly"""
    with patch('requests.get') as mock_get:
        # Arrange
        mock_response = Mock()
        mock_response.raise_for_status.side_effect = requests.HTTPError("404 Not Found")
        mock_get.return_value = mock_response

        client = APIClient("https://api.example.com")

        # Act & Assert
        with pytest.raises(requests.HTTPError):
            client.get_user("nonexistent")

print("Running API client mock tests...")
test_api_client_get_user()
test_api_client_create_user()
test_api_client_handles_http_errors()
print("API client mock tests passed!")

In [ ]:
# Environment variable mocking
def test_config_service_with_env_vars():
    """Test config service with environment variables"""
    with patch.dict(os.environ, {'DATABASE_URL': 'postgresql://test', 'API_KEY': 'test_key_123'}):
        # Test database URL
        db_url = ConfigService.get_database_url()
        assert db_url == 'postgresql://test'

        # Test API key
        api_key = ConfigService.get_api_key()
        assert api_key == 'test_key_123'

def test_config_service_defaults():
    """Test config service with default values"""
    with patch.dict(os.environ, {}, clear=True):  # Clear all env vars
        # Test default database URL
        db_url = ConfigService.get_database_url()
        assert db_url == 'sqlite:///default.db'

        # Test missing API key raises error
        with pytest.raises(ValueError, match="API_KEY environment variable is required"):
            ConfigService.get_api_key()

def test_config_service_partial_env():
    """Test config service with partial environment variables"""
    with patch.dict(os.environ, {'API_KEY': 'partial_key'}, clear=True):
        # Database URL should use default
        db_url = ConfigService.get_database_url()
        assert db_url == 'sqlite:///default.db'

        # API key should be available
        api_key = ConfigService.get_api_key()
        assert api_key == 'partial_key'

print("Running config service mock tests...")
test_config_service_with_env_vars()
test_config_service_defaults()
test_config_service_partial_env()
print("Config service mock tests passed!")

In [ ]:
# Advanced mocking: Spy pattern and call verification
class UserService:
    """Service that coordinates user operations"""

    def __init__(self, api_client: APIClient, email_service: EmailService):
        self.api_client = api_client
        self.email_service = email_service

    def create_user_and_notify(self, user_data: Dict[str, Any]) -> str:
        """Create user and send welcome email"""
        # Create user
        user_id = self.api_client.create_user(user_data)

        # Send welcome email
        welcome_sent = self.email_service.send_email(
            "noreply@example.com",
            user_data['email'],
            "Welcome!",
            f"Welcome {user_data['name']}! Your user ID is {user_id}."
        )

        if not welcome_sent:
            # Log the error but don't fail user creation
            print(f"Warning: Failed to send welcome email to {user_data['email']}")

        return user_id

def test_user_service_integration():
    """Test user service with multiple mocked dependencies"""
    # Arrange
    mock_api_client = Mock(spec=APIClient)
    mock_email_service = Mock(spec=EmailService)

    # Configure mocks
    mock_api_client.create_user.return_value = "user_456"
    mock_email_service.send_email.return_value = True

    user_service = UserService(mock_api_client, mock_email_service)
    user_data = {'name': 'Integration User', 'email': 'integration@example.com'}

    # Act
    user_id = user_service.create_user_and_notify(user_data)

    # Assert
    assert user_id == "user_456"

    # Verify API client was called correctly
    mock_api_client.create_user.assert_called_once_with(user_data)

    # Verify email service was called correctly
    mock_email_service.send_email.assert_called_once_with(
        "noreply@example.com",
        "integration@example.com",
        "Welcome!",
        "Welcome Integration User! Your user ID is user_456."
    )

def test_user_service_email_failure():
    """Test user service when email sending fails"""
    # Arrange
    mock_api_client = Mock(spec=APIClient)
    mock_email_service = Mock(spec=EmailService)

    # Configure mocks
    mock_api_client.create_user.return_value = "user_789"
    mock_email_service.send_email.return_value = False  # Email fails

    user_service = UserService(mock_api_client, mock_email_service)
    user_data = {'name': 'Test User', 'email': 'test@example.com'}

    # Act
    user_id = user_service.create_user_and_notify(user_data)

    # Assert
    assert user_id == "user_789"  # User creation should still succeed

    # Verify both services were called
    mock_api_client.create_user.assert_called_once()
    mock_email_service.send_email.assert_called_once()

print("Running integration mock tests...")
test_user_service_integration()
test_user_service_email_failure()
print("Integration mock tests passed!")

## Key Takeaways

1. **AAA Pattern**: Provides clear test structure - Arrange setup, Act execution, Assert verification
2. **Fixtures**: Reduce code duplication and ensure consistent test environments
3. **Mocking**: Isolates units under test by replacing external dependencies
4. **Test Isolation**: Each test should be independent and not affect others

## Best Practices

- Use descriptive test names that explain what is being tested
- Keep tests simple and focused on single behaviors
- Use appropriate test doubles (mocks, stubs, fakes) for different scenarios
- Always clean up resources in fixtures
- Verify both positive and negative test cases

## Exercises

1. Write tests for a `Calculator` class using the AAA pattern
2. Create pytest fixtures for testing a file processing system
3. Mock a database connection to test a repository pattern implementation
4. Use environment variable mocking to test configuration-dependent code
5. Practice testing exception handling scenarios with mocks